In [17]:
import cv2
import math
import numpy as np

In [18]:
def preprocess(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blur, 50, 150)
    return edges


In [19]:
def roi(img):
    h, w = img.shape
    triangle = np.array([
        [(50, h), (w - 50, h), (w//2, h//2)]
    ])

    mask = np.zeros_like(img)
    cv2.fillPoly(mask, triangle, 255)
    return cv2.bitwise_and(img, mask)



In [20]:
def average_slope(frame, lines):
    left_fit = []
    right_fit = []
    
    if lines is None:
        return None
    
    for line in lines:
        x1, y1, x2, y2 = line[0]
        if x2 == x1:
            continue
        
        slope = (y2 - y1) / (x2 - x1)
        
        intercept = y1 - slope * x1
        if slope < 0:
            left_fit.append((slope, intercept))
        else:
            right_fit.append((slope, intercept))
            
    left_line = make_line(frame, np.mean(left_fit, axis = 0)) if left_fit else None
    right_line = make_line(frame, np.mean(right_fit, axis = 0)) if right_fit else None
    
    return left_line, right_line

In [21]:
def make_line(frame, line):
    slope, intercept = line
    y1 = frame.shape[0]
    y2 = int(y1 * 0.6)
    
    if slope == 0 or np.isinf(slope):
        return None
    x1 = int((y1 - intercept) / slope)
    x2 = int((y2 - intercept) / slope)
    return [[x1, y1, x2, y2]]

In [22]:
def detect_line(frame, lines):

    line_img = np.zeros_like(frame)
    for line in lines:
        if line is not None:
            x1, y1, x2, y2 = line[0]
            cv2.line(line_img, (x1, y1), (x2, y2), (0, 255, 0), 5)
    return line_img

In [23]:
def calculate_steering_angle(frame, left_line, right_line):
    h, w, _ = frame.shape
    if left_line is None or right_line is None:
        return 0
    
    _, _, left_x2 , _ = left_line[0]
    _, _, right_x2, _ = right_line[0]
    
    mid_x = (left_x2 + right_x2) // 2
    car_mid = w // 2
    
    dx = mid_x - car_mid
    dy = h * 0.6
    
    angle_rad = math.atan2(dx, dy)
    angle_deg = int(angle_rad * 180.0/ math.pi)
    
    return angle_deg

In [24]:
def display_heading_line(frame, steering_angle):
    h, w, _ = frame.shape
    arrow_length = 100
    
    angle_rad = math.radians(steering_angle)
    x1, y1 = w // 2, h
    x2 = int(x1 - arrow_length * math.sin(angle_rad))
    y2 = int(y1 - arrow_length * math.cos(angle_rad))
    
    cv2.arrowedLine(frame, (x1, y1), (x2, y2), (255, 0, 0), 5)
    return frame

In [27]:
cap = cv2.VideoCapture("video2.mp4")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    edges = preprocess(frame)
    region = roi(edges)
    
    lines = cv2.HoughLinesP(region, 2, np.pi/180, 100,
                            np.array([]), minLineLength=40, maxLineGap=5)
    
    left_line, right_line = average_slope(frame, lines)
    lane_lines = detect_line(frame, [left_line, right_line])
    combo = cv2.addWeighted(frame, 0.8, lane_lines, 1, 1)
    
    steering_angle = calculate_steering_angle(frame, left_line, right_line)
    
    combo = display_heading_line(combo, steering_angle)
    
    cv2.imshow('Lane Detection', combo)
    
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()

In [28]:
# lane_detection_full.py
import cv2
import numpy as np
import math

# --------------------
# 1. Region of Interest
# --------------------
def region_of_interest(img):
    height, width = img.shape
    polygons = np.array([
        [(50, height), (width-50, height), (width//2, int(height*0.6))]
    ])
    mask = np.zeros_like(img)
    cv2.fillPoly(mask, polygons, 255)
    return cv2.bitwise_and(img, mask)

# --------------------
# 2. Make Line Points (from slope, intercept)
# --------------------
def make_line_points(frame, line):
    slope, intercept = line
    # protect against zero slope
    if slope == 0:
        return None
    y1 = frame.shape[0]
    y2 = int(y1 * 0.6)
    x1 = int((y1 - intercept) / slope)
    x2 = int((y2 - intercept) / slope)
    return [[x1, y1, x2, y2]]

# --------------------
# 3. Average slopes for left + right lanes
# --------------------
def average_slope_intercept(frame, lines):
    left_fit, right_fit = [], []
    if lines is None:
        return None, None

    for line in lines:
        x1, y1, x2, y2 = line[0]
        if x2 == x1:  # vertical line, skip
            continue
        slope = (y2 - y1) / (x2 - x1)
        intercept = y1 - slope * x1
        # small slope filtering to avoid horizontal-ish spurious lines
        if abs(slope) < 0.3:
            continue
        if slope < 0:
            left_fit.append((slope, intercept))
        else:
            right_fit.append((slope, intercept))

    left_line = None
    right_line = None
    if left_fit:
        left_avg = np.mean(left_fit, axis=0)
        left_line = make_line_points(frame, left_avg)
    if right_fit:
        right_avg = np.mean(right_fit, axis=0)
        right_line = make_line_points(frame, right_avg)

    return left_line, right_line

# --------------------
# 4. Draw lines
# --------------------
def display_lines(frame, lines, thickness=10):
    line_img = np.zeros_like(frame)
    if lines is None:
        return line_img
    for line in lines:
        if line is None:
            continue
        x1, y1, x2, y2 = line[0]
        cv2.line(line_img, (x1, y1), (x2, y2), (0, 255, 0), thickness)
    return line_img

# --------------------
# 5. Fill lane area
# --------------------
def fill_lane_area(frame, left_line, right_line, alpha=0.3):
    overlay = np.zeros_like(frame)
    if left_line is None or right_line is None:
        return frame
    # build polygon points: bottom left, top left, top right, bottom right
    pts = np.array([[
        (left_line[0][0], left_line[0][1]),
        (left_line[0][2], left_line[0][3]),
        (right_line[0][2], right_line[0][3]),
        (right_line[0][0], right_line[0][1])
    ]], dtype=np.int32)
    cv2.fillPoly(overlay, pts, (0, 255, 0))
    return cv2.addWeighted(frame, 1.0, overlay, alpha, 0)

# --------------------
# 6. Steering angle (deg) from lane midpoint
# --------------------
def calculate_steering_angle(frame, left_line, right_line):
    height, width, _ = frame.shape
    if left_line is None or right_line is None:
        return 0
    left_x2 = left_line[0][2]
    right_x2 = right_line[0][2]
    mid_x = (left_x2 + right_x2) // 2
    car_mid = width // 2
    dx = mid_x - car_mid
    dy = height * 0.6  # lookahead
    angle_rad = math.atan2(dx, dy)
    angle_deg = int(angle_rad * 180.0 / math.pi)
    return angle_deg

# --------------------
# 7. Draw steering arrow
# --------------------
def display_heading_line(frame, steering_angle, length=150):
    out = frame.copy()
    height, width, _ = frame.shape
    angle_rad = math.radians(steering_angle)
    x1, y1 = width // 2, height
    x2 = int(x1 - length * math.sin(angle_rad))
    y2 = int(y1 - length * math.cos(angle_rad))
    cv2.arrowedLine(out, (x1, y1), (x2, y2), (255, 0, 0), 6, tipLength=0.3)
    return out

# --------------------
# 8. Curvature radius via poly fit (pixels)
# --------------------
def calculate_curvature(left_line, right_line, frame_height):
    # need at least two points on each to fit; here we use endpoints
    if left_line is None or right_line is None:
        return None
    left_x = [left_line[0][0], left_line[0][2]]
    left_y = [left_line[0][1], left_line[0][3]]
    right_x = [right_line[0][0], right_line[0][2]]
    right_y = [right_line[0][1], right_line[0][3]]
    x = np.array(left_x + right_x)
    y = np.array(left_y + right_y)
    # fit x = A*y^2 + B*y + C
    if len(x) < 3 or len(y) < 3:
        return None
    try:
        fit = np.polyfit(y, x, 2)  # returns [A, B, C]
    except np.RankWarning:
        return None
    A, B, _ = fit
    y_eval = frame_height
    denom = 2 * A
    if denom == 0:
        return None
    curvature_radius = ((1 + (2*A*y_eval + B)**2)**1.5) / abs(denom)
    return curvature_radius

# --------------------
# 9. Lane departure warning
# --------------------
def check_lane_departure(frame, left_line, right_line, threshold_px=50):
    h, w, _ = frame.shape
    if left_line is None or right_line is None:
        return "No Lane Detected", False
    left_x_bottom = left_line[0][0]
    right_x_bottom = right_line[0][0]
    lane_center = (left_x_bottom + right_x_bottom) // 2
    car_center = w // 2
    deviation = car_center - lane_center
    if abs(deviation) > threshold_px:
        return "⚠ Lane Departure!", True
    else:
        return "On Lane", False

# --------------------
# 10. Main pipeline
# --------------------
def main(video_path="video2.mp4"):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("ERROR: cannot open video:", video_path)
        return

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # preprocess
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray, (5,5), 0)
        edges = cv2.Canny(blur, 50, 150)

        # mask
        cropped = region_of_interest(edges)

        # detect lines
        lines = cv2.HoughLinesP(cropped, 2, np.pi/180, 100, np.array([]),
                                minLineLength=40, maxLineGap=5)

        # average & make smooth lines
        left_line, right_line = average_slope_intercept(frame, lines)

        # draw lanes
        lane_lines_img = display_lines(frame, [left_line, right_line], thickness=10)
        combo = cv2.addWeighted(frame, 0.8, lane_lines_img, 1, 1)

        # fill lane area
        combo = fill_lane_area(combo, left_line, right_line, alpha=0.25)

        # steering angle & display arrow
        steering_angle = calculate_steering_angle(frame, left_line, right_line)
        combo = display_heading_line(combo, steering_angle, length=150)

        # curvature
        curvature = calculate_curvature(left_line, right_line, frame.shape[0])
        if curvature is not None and not math.isinf(curvature) and not math.isnan(curvature):
            cv2.putText(combo, f"Curvature: {int(curvature)} px", (40, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,255,255), 2)

        # lane departure
        warning_text, is_departure = check_lane_departure(frame, left_line, right_line, threshold_px=50)
        color = (0,0,255) if is_departure else (0,255,0)
        cv2.putText(combo, warning_text, (40, 100),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

        # steering angle numeric
        cv2.putText(combo, f"Steer: {steering_angle} deg", (40, 140),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (200,200,0), 2)

        cv2.imshow("Lane Detection + ADAS", combo)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main("test_video.mp4")


ERROR: cannot open video: test_video.mp4
